# Apply AI Superresolution to video - Prediction Notebook


# Project Documentation

In [1]:

from pathlib import Path
import os


project_base = "/media/SSD/superres/"
results_folder = "Results"
data_folder = "video_data"
model_folder = "checkpoint"

model_name_base = "NEWCNN5"


## Experiment documentation

In [2]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [3]:
print("NOTEBOOK_FULL_PATH:\n", NOTEBOOK_FULL_PATH)

NOTEBOOK_FULL_PATH:
 /SSD_link/superres/Superresolution train New Network with kernel predictor make predictions.ipynb


In [4]:
nb_path = Path(NOTEBOOK_FULL_PATH)
print(nb_path)

# Make result folder if it does not exist
result_path = Path(project_base, results_folder , nb_path.stem )
print(result_path)
os.makedirs(result_path, exist_ok=True)

/SSD_link/superres/Superresolution train New Network with kernel predictor make predictions.ipynb
/media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions


In [5]:
model_out_path=Path(project_base)/model_folder
print(model_out_path)
os.makedirs(model_out_path, exist_ok=True)

/media/SSD/superres/checkpoint


In [6]:

#%reload_ext autoreload
#%autoreload 2
#%matplotlib inline


import cv2 
import numpy as np
import subprocess as sp
import time
from tqdm import tqdm, trange
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from torchvision.transforms import RandomHorizontalFlip,ColorJitter
from torchvision.transforms import Compose, CenterCrop, ToTensor, Resize, Grayscale,ToPILImage,Normalize

import random

import lpips #https://github.com/richzhang/PerceptualSimilarity

import torch.utils.data as data

from os import listdir
from os.path import join
from PIL import Image, ImageFilter
import numpy as np
import torchvision as vision

import numbers



from torch.cuda.amp import autocast

torch.cuda.manual_seed(42)


## Create a data loader
- see https://github.com/2KangHo/vdsr_pytorch/blob/master/data.py
- see https://github.com/2KangHo/vdsr_pytorch/blob/master/data_utils.py

In [7]:
from superres_utils import make_framelist, load_frame, RescaleCrop, Multifuz, DatasetFromVideoFolder
from superres_utils import show_batch
from superres_utils import pil_disk_blur, display_PSNR, test_results, superres_mp4

In [8]:
def input_transform():        
    return Compose([        
        #Grayscale(num_output_channels=1),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])



def target_transform(crop_size):
    return Compose([
        #Grayscale(num_output_channels=1),
        CenterCrop(crop_size),
        ToTensor(),
    ])


In [ ]:
valid_dir = "/home/filip/SSD_link/superres/DIV2K/DIV2K_valid_HR"
#frame_sample_size =10
#batch_size = 4 # Make 4 For FP16, 480x720 size images
#crop_size = (480,720)

kernel_range=(1,16)
quality_range=(30,99) 
slope_range=(0.001, 0.1)
radius_range=(0,0)  # make all one fuzziness
scale_range=(1/2,2)
Flip_hor=False
Rand_bright_contrast=False
match_exp = "*.*"

In [ ]:
valid_set = DatasetFromVideoFolder(valid_dir,crop_size,frame_sample_size,
                                   input_transform=input_transform(),
                                   target_transform=target_transform(crop_size),
                                   Flip_hor=Flip_hor,
                                   Rand_bright_contrast=Rand_bright_contrast,
                                   kernel_range=kernel_range,
                                   quality_range=quality_range,
                                   slope_range=slope_range,
                                   radius_range=radius_range,
                                   scale_range=scale_range,
                                   max_kernel=10,
                                   match_exp=match_exp,
                            )
valid_data_loader = DataLoader(dataset=valid_set, num_workers=0, batch_size=batch_size, shuffle=False)

In [ ]:
show_batch(valid_data_loader,size =8)

# NEW CNN

In [ ]:
import torch.nn as nn

class NEWNet5(nn.Module):
    def __init__(self, c1, c2, c3):
        super(NEWNet5, self).__init__()   
        
        self.p1 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=c1, kernel_size=3, stride=1, padding=1, bias=False),
            nn.PReLU(),
            nn.Conv2d(in_channels=c1, out_channels=c1, kernel_size=3, stride=1, padding=1, bias=True),
            nn.PReLU(),
            nn.Conv2d(in_channels=c1, out_channels=c1, kernel_size=3, stride=1, padding=1, bias=True),
            nn.PReLU()
        )
        
        self.p2 = nn.Sequential(
            nn.Conv2d(in_channels=c1, out_channels=c2, kernel_size=1, stride=1, padding=0, bias=True),
            nn.PReLU(),
            nn.Conv2d(in_channels=c2, out_channels=c2, kernel_size=3, stride=1, padding=1, bias=True),
            nn.PReLU(),
            nn.Conv2d(in_channels=c2, out_channels=c1, kernel_size=1, stride=1, padding=0, bias=True)
        )
        
        

        
        self.prelu = nn.PReLU()
        self.convf = nn.Conv2d(in_channels=c1, out_channels=3, kernel_size=3, stride=1, padding=1, bias=True)
        
        
        

              
        
        
    def forward(self, x, k):    
        
        residual = x
        # add insert k as the 4th channel into x, which has three channels
        one_channel = x[:,0,:,:].unsqueeze(1)  #use one channel as a template
        t1 = torch.ones_like(one_channel.permute(3,2,1,0))  #make a ones tensor like the channel
        res = k[:,0]*t1  # multiply by k, so that the batch spreads to the size of an image
        res = res.permute(3,2,1,0) # put back in proper order
        xk = torch.cat((x,res), dim=1) #append the 4th channel
        
        
        
        
        xk = self.p1(xk)
        skip1 = xk
                        
        xk = self.p2(xk)
        xk = xk.add_(skip1)
        xk = self.prelu(xk)
        xk = self.convf(xk)
        out = torch.add(xk,residual)
        return out

# VDSR 

In [11]:
import sys
sys.path.insert(0, '/media/SSD/superres/pytorch-vdsr/')

from vdsr import Net

# New models - Load the models

In [ ]:
kmodel = torch.load("/media/SSD/superres/checkpoint/BLUR_SHUFFLE1_epoch_18.pth")["model"]

In [ ]:
kmodel.cuda()
kmodel.eval();

In [ ]:
model = torch.load("/media/SSD/superres/checkpoint/NEWCNN5_epoch_54.pth")["model"]

In [ ]:
class Combine_Nets(nn.Module):
    def __init__(self, model, kmodel):
        super(Combine_Nets, self).__init__()   
        
        
        self.model = model
        self.kmodel = kmodel
        
        

    def forward(self, x):    
        
       
        k = self.kmodel(x)
        x = self.model(x,k)
        
        return x,k

In [ ]:
mcombine = Combine_Nets(model,kmodel)

## VDSR load model

In [12]:
model = torch.load("/media/SSD/superres/checkpoint/hresperfuzmodel2_epoch_12.pth")["model"]

In [13]:
model

Net(
  (residual_layer): Sequential(
    (0): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (1): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (2): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (3): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (4): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (5): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (6): Conv_ReLU_Bl

In [14]:
def cv2_to_batch_tensor(img_bgr):
    mn=np.array([0.485, 0.456, 0.406])
    std=np.array([0.229, 0.224, 0.225])
    im_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    im_rgb = im_rgb/255.0
    im_rgb = (im_rgb-mn)/std
    tform = im_rgb.transpose(( 2,0, 1))
    tform = torch.from_numpy(tform).float()
    tform = tform.unsqueeze(0)
    return tform

In [15]:
def batch_tensor_to_cv2(bt_3ch):
    bt_3ch = (bt_3ch*255.0).clamp(0,255)
    t_3ch = bt_3ch.squeeze(0)
    #t_3ch = t_3ch.cpu()
    im_rbg = t_3ch.permute(1,2,0).numpy().astype(np.uint8)
    im_bgr = cv2.cvtColor(im_rbg, cv2.COLOR_RGB2BGR)
    
    return im_bgr

In [ ]:
#test
#img_cv = cv2.imread("/media/SSD/superres/Screenshot from snip.mp4.png")
img_cv = cv2.imread("/home/filip/Pictures/Screenshot from sup8snip.mp4.png")



inputs = cv2_to_batch_tensor(img_bgr=img_cv)
inputs = inputs.cuda()

            
with torch.no_grad():
    with autocast():
        out = mcombine(inputs)
im_out = batch_tensor_to_cv2(out[0].cpu())
plt.imshow(cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB))
print("k=",out[1][0].item())

In [ ]:
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

In [ ]:
#test
#img_cv = cv2.imread("/media/SSD/superres/Screenshot from snip.mp4.png")
img_cv = cv2.imread("/home/filip/Pictures/Screenshot from sup8snip.mp4.png")



inputs = cv2_to_batch_tensor(img_bgr=img_cv)
inputs = inputs.cuda()

k=torch.tensor([[6.28]]).cuda()
            
with torch.no_grad():
    with autocast():
        k2 = kmodel(inputs)
        out = model(inputs, k)
im_out = batch_tensor_to_cv2(out[0].cpu())


sharpened_image = unsharp_mask(img_cv, kernel_size=(5, 5), sigma=1.0, amount=2.5, threshold=0)


fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(20, 20))

axes[0].imshow(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
axes[0].axis('off')
axes[1].imshow(cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB))
axes[1].axis('off')
axes[2].imshow(cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2RGB))
axes[2].axis('off')
plt.tight_layout()
plt.show()

print("k=",k2)

# convert sample videos

In [23]:
from superres_utils import decolorize_cv, colorize_cv, cv_center_crop

from torch.cuda.amp import autocast
def superres_mp4(infile, outfile, model,channels=1, factor=1.0,display=True,
                 bitrate= "12000k",errorfile = None, k_scale = 1, center_crop = None, add_k_bar = False):
    
    mn=np.array([0.485, 0.456, 0.406])
    std=np.array([0.229, 0.224, 0.225])
    
    model = model.cuda()
    #torch.set_grad_enabled(False)
    model.eval()
    
    cap = cv2.VideoCapture(str(infile))
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    
    if center_crop is None:
        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
    else:
        width,height = center_crop
    
    
    success_flag = False
    
    print("Original File:", str(infile))
    print("frames=",length,"\nwidth=",width,"\nheight=",height,"\nfps=",fps)
    
    

    new_height = int(height*factor)
    new_width = int(width*factor)
    
    print("\nProcess File:", str(outfile))
    print("factor:",factor,"\nwidth=",new_width, "\nheight=",new_height,"\nbitrate=",bitrate)
    
    

    dimension = '{}x{}'.format(new_width, new_height)  #ffmpeg uses bicubic as default scaling alg
    f_format = 'bgr24' # remember OpenCV uses bgr format




    command = ['ffmpeg',
            '-y',
            '-f', 'rawvideo',
            '-vcodec','rawvideo',
            '-s', dimension,
            '-pix_fmt', 'bgr24',
            '-r', str(fps),
            '-i', '-',
            '-i', str(infile),
            '-c:v', 'h264',
            '-c:a', 'aac',

            '-map','0:v:0',
            '-map','1:a:0',
            '-shortest',
            '-b:v', bitrate, 
            str(outfile) ]


    if errorfile is not None:
        ef = open(errorfile,"w+")
        p = sp.Popen(command, stdin=sp.PIPE, stderr=ef)
    else:
        p = sp.Popen(command, stdin=sp.PIPE)

    # Full processing with a stream instead of a temp file for video
    k_preds = []
    pbar = tqdm(total=length)
    while(cap.isOpened()):




        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            
            #crop frame if needed
            if center_crop is not None:
                frame = cv_center_crop(frame,(height,width))
            
            if (factor != 1.0):
                frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)
                #frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
            
            if channels == 1:
                im_input, im_b_ycbcr = decolorize_cv(frame)

                im_input = Variable(torch.from_numpy(im_input).float()).view(1, -1, im_input.shape[0], im_input.shape[1])
                im_input = im_input.cuda()

                with torch.no_grad():
                    with autocast():
                        mout = model(im_input)
                        
                if len(mout) >1:
                    out=mout[0].cpu()
                    
                else:
                    out = mout.cpu()
                    
                out = (out*255.0).clamp(0,255)
                

                im_h_y = out.data[0].numpy().astype(np.float32)

                im_h_y = im_h_y[0,:,:]

                im_h = colorize_cv(im_h_y, im_b_ycbcr)
            elif channels == 3:
                im_input = cv2_to_batch_tensor(frame)
        
  
                im_input = im_input.cuda()
    
                
                
                with torch.no_grad():
                    with autocast():
                        mout = model(im_input)  
                
                if len(mout) >1:
                    out=mout[0].cpu()
                    
                else:
                    out = mout.cpu()
                    
                
                im_h = batch_tensor_to_cv2(out)
                

            #Add notation and crude bar chart to video frame
            
            if add_k_bar and len(mout)>1:
                cv2.putText(im_h, text='k={:.2f}'.format(k_scale*mout[1].item()), org=(new_width-70,new_height-50),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0,0,255),
                thickness=1, lineType=cv2.LINE_AA)
                
                cv2.putText(im_h, text='_', org=(new_width-5,new_height - int(new_height*mout[1].item()/17)),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0,0,255),
                thickness=2, lineType=cv2.LINE_AA)
                
                
            p.stdin.write(im_h.tobytes())


            if display:
                cv2.imshow('Processed',im_h)
                time.sleep(10)
                #cv2.imshow('Orig',frame)
            pbar.update(1)
            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                success_flag = False
                break
        # Break the loop
        else:
            success_flag = True
            break
    # When everything done, release the video capture object

    pbar.close()

    p.stdin.close()
    p.wait()

    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()
    
    return success_flag

In [17]:
result_path

PosixPath('/media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions')

In [18]:
from pytube import YouTube
import os
import shutil
import math
import datetime

In [19]:
# Video 1

In [20]:
infile = YouTube('https://www.youtube.com/watch?v=NjDclfAFRB4').streams.get_highest_resolution().download('/media/SSD/superres/video_data/demos')

In [51]:
factor = 1.0
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSR.mp4"
outfile = result_path/fn

outfile

PosixPath('/media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Late 1890s - A Trip Through Paris France (speed corrected w added sound)-10VDSR.mp4')

In [24]:
superres_mp4(infile, outfile, model,channels=1, factor=factor,display=False,bitrate= "4000k", add_k_bar=True)


  0%|          | 0/10739 [00:00<?, ?it/s]

Original File: /media/SSD/superres/video_data/demos/Late 1890s - A Trip Through Paris France (speed corrected w added sound).mp4
frames= 10739 
width= 1280 
height= 720 
fps= 29.97002997002997

Process File: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Late 1890s - A Trip Through Paris France (speed corrected w added sound)-10VDSR.mp4
factor: 1.0 
width= 1280 
height= 720 
bitrate= 4000k



  3%|▎         | 298/10739 [00:41<24:27,  7.12it/s]


  6%|▌         | 594/10739 [01:21<23:01,  7.34it/s]


  8%|▊         | 890/10739 [02:02<21:49,  7.52it/s]


...

 88%|████████▊ | 9503/10739 [18:23<02:04,  9.95it/s]


 93%|█████████▎| 9940/10739 [19:07<01:19, 10.09it/s]


 96%|█████████▋| 10351/10739 [19:48<00:39,  9.88it/s]


100%|██████████| 10739/10739 [20:26<00:00,  8.75it/s]


True

In [52]:
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSRdual.mp4"
dualfile_s = str(result_path/fn)
infile_s = str(infile)
outfile_s = str(outfile)
print("Input 1:",infile_s)
print("Input 2:",outfile_s)
print("Output:",dualfile_s)

Input 1: /media/SSD/superres/video_data/demos/Late 1890s - A Trip Through Paris France (speed corrected w added sound).mp4
Input 2: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Late 1890s - A Trip Through Paris France (speed corrected w added sound)-10VDSR.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Late 1890s - A Trip Through Paris France (speed corrected w added sound)-10VDSRdual.mp4


In [53]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile_s+'"'} -i {'"'+outfile_s+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile_s+'"'}

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/media/SSD/conda_env/superres --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/media/SSD/su

In [25]:
#Video 2

In [26]:
factor = 1.0
addr = 'https://www.youtube.com/watch?v=PSStsZCNAMY'
infile = YouTube(addr).streams.get_highest_resolution().download('/media/SSD/superres/video_data/demos')
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSR.mp4"
outfile = result_path/fn
print("Input:", infile)
print("Output:",outfile)

Input: /media/SSD/superres/video_data/demos/8mm Film Restored 1939-1940.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-10VDSR.mp4


In [27]:
#superres_mp4(infile, outfile, mcombine,channels=3, factor=factor,display=False,bitrate= "4000k")
superres_mp4(infile, outfile, model,channels=1, factor=factor,display=False,bitrate= "4000k", add_k_bar=True)


  0%|          | 5/4204 [00:00<01:30, 46.52it/s]

Original File: /media/SSD/superres/video_data/demos/8mm Film Restored 1939-1940.mp4
frames= 4204 
width= 480 
height= 360 
fps= 30.0

Process File: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-10VDSR.mp4
factor: 1.0 
width= 480 
height= 360 
bitrate= 4000k



 20%|██        | 860/4204 [00:17<01:09, 48.18it/s]


 59%|█████▊    | 2467/4204 [00:50<00:35, 49.05it/s]


 97%|█████████▋| 4087/4204 [01:23<00:02, 51.96it/s]


100%|██████████| 4204/4204 [01:25<00:00, 49.06it/s]


True

In [28]:
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSRdual.mp4"
dualfile_s = str(result_path/fn)
infile_s = str(infile)
outfile_s = str(outfile)
print("Input 1:",infile_s)
print("Input 2:",outfile_s)
print("Output:",dualfile_s)

Input 1: /media/SSD/superres/video_data/demos/8mm Film Restored 1939-1940.mp4
Input 2: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-10VDSR.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-10VDSRdual.mp4


In [29]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile_s+'"'} -i {'"'+outfile_s+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile_s+'"'}

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/media/SSD/conda_env/superres --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/media/SSD/su

In [30]:
# Video 3

In [31]:
factor = 2.0
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSR.mp4"
outfile = result_path/fn
print("Input:", infile)
print("Output:",outfile)

Input: /media/SSD/superres/video_data/demos/8mm Film Restored 1939-1940.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-20VDSR.mp4


In [32]:
#superres_mp4(infile, outfile, mcombine,channels=3, factor=factor,display=False,bitrate= "4000k")
superres_mp4(infile, outfile, model,channels=1, factor=factor,display=False,bitrate= "4000k", add_k_bar=True)

Original File: /media/SSD/superres/video_data/demos/8mm Film Restored 1939-1940.mp4
frames= 4204 
width= 480 
height= 360 
fps= 30.0

Process File: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-20VDSR.mp4
factor: 2.0 
width= 960 
height= 720 
bitrate= 4000k



 14%|█▍        | 603/4204 [00:45<04:59, 12.01it/s]


 29%|██▊       | 1203/4204 [01:32<03:49, 13.06it/s]


 43%|████▎     | 1795/4204 [02:17<03:02, 13.20it/s]


 57%|█████▋    | 2387/4204 [03:03<02:19, 13.03it/s]


 71%|███████   | 2979/4204 [03:48<01:33, 13.09it/s]


 85%|████████▍ | 3571/4204 [04:33<00:48, 13.11it/s]


 99%|█████████▉| 4163/4204 [05:19<00:03, 13.08it/s]


100%|██████████| 4204/4204 [05:22<00:00, 13.04it/s]


True

In [33]:
# Video 4

In [34]:
factor = 1.5
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSR.mp4"
outfile = result_path/fn
print("Input:", infile)
print("Output:",outfile)

Input: /media/SSD/superres/video_data/demos/8mm Film Restored 1939-1940.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-15VDSR.mp4


In [35]:
#superres_mp4(infile, outfile, mcombine,channels=3, factor=factor,display=False,bitrate= "4000k")
superres_mp4(infile, outfile, model,channels=1, factor=factor,display=False,bitrate= "4000k", add_k_bar=True)


  0%|          | 0/4204 [00:00<?, ?it/s]

Original File: /media/SSD/superres/video_data/demos/8mm Film Restored 1939-1940.mp4
frames= 4204 
width= 480 
height= 360 
fps= 30.0

Process File: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-15VDSR.mp4
factor: 1.5 
width= 720 
height= 540 
bitrate= 4000k



 22%|██▏       | 909/4204 [00:40<02:27, 22.33it/s]


 43%|████▎     | 1800/4204 [01:20<01:45, 22.74it/s]


 64%|██████▍   | 2688/4204 [01:59<01:07, 22.49it/s]


 85%|████████▌ | 3576/4204 [02:39<00:28, 22.21it/s]


100%|██████████| 4204/4204 [03:07<00:00, 22.46it/s]


True

In [36]:
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSRdual.mp4"
dualfile_s = str(result_path/fn)
infile_s = str(infile)
outfile_s = str(outfile)
print("Input 1:",infile_s)
print("Input 2:",outfile_s)
print("Output:",dualfile_s)

Input 1: /media/SSD/superres/video_data/demos/8mm Film Restored 1939-1940.mp4
Input 2: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-15VDSR.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-15VDSRdual.mp4


In [37]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile_s+'"'} -i {'"'+outfile_s+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile_s+'"'}

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/media/SSD/conda_env/superres --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/media/SSD/su

In [38]:
#Video 5

In [39]:
factor = 1.0
addr = 'https://www.youtube.com/watch?v=QAVt-HROuIU'
infile = YouTube(addr).streams.get_highest_resolution().download('/media/SSD/superres/video_data/demos')
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSR.mp4"
outfile = result_path/fn
print("Input:", infile)
print("Output:",outfile)

Input: /media/SSD/superres/video_data/demos/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker-10VDSR.mp4


In [40]:
#superres_mp4(infile, outfile, mcombine,channels=3, factor=factor,display=False,bitrate= "4000k")
superres_mp4(infile, outfile, model,channels=1, factor=factor,display=False,bitrate= "4000k", add_k_bar=True)


  0%|          | 0/8509 [00:00<?, ?it/s]

Original File: /media/SSD/superres/video_data/demos/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker.mp4
frames= 8509 
width= 1280 
height= 720 
fps= 29.97002997002997

Process File: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker-10VDSR.mp4
factor: 1.0 
width= 1280 
height= 720 
bitrate= 4000k



  4%|▍         | 334/8509 [00:36<14:53,  9.15it/s]


  8%|▊         | 679/8509 [01:13<14:01,  9.31it/s]


 12%|█▏        | 1018/8509 [01:49<13:27,  9.28it/s]


...

 84%|████████▎ | 7111/8509 [12:10<02:18, 10.11it/s]


 90%|█████████ | 7675/8509 [13:06<01:22, 10.10it/s]


 97%|█████████▋| 8257/8509 [14:04<00:24, 10.11it/s]


100%|██████████| 8509/8509 [14:29<00:00,  9.79it/s]


True

In [41]:
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSRdual.mp4"
dualfile_s = str(result_path/fn)
infile_s = str(infile)
outfile_s = str(outfile)
print("Input 1:",infile_s)
print("Input 2:",outfile_s)
print("Output:",dualfile_s)

Input 1: /media/SSD/superres/video_data/demos/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker.mp4
Input 2: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker-10VDSR.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker-10VDSRdual.mp4


In [42]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile_s+'"'} -i {'"'+outfile_s+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile_s+'"'}

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/media/SSD/conda_env/superres --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/media/SSD/su

In [43]:
#Video 6

In [44]:
factor = 1.0
#addr = 'https://www.youtube.com/watch?v=QAVt-HROuIU'
#infile = YouTube(addr).streams.get_highest_resolution().download('/media/SSD/superres/video_data/demos')
infile = "/media/SSD/superres/super8/super8_best_from_mp2.mp4"
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSR.mp4"
outfile = result_path/fn
print("Input:", infile)
print("Output:",outfile)

Input: /media/SSD/superres/super8/super8_best_from_mp2.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/super8_best_from_mp2-10VDSR.mp4


In [45]:
#superres_mp4(infile, outfile, mcombine,channels=3, factor=factor,display=False,bitrate= "4000k")
superres_mp4(infile, outfile, model,channels=1, factor=factor,display=False,bitrate= "4000k", add_k_bar=True)

Original File: /media/SSD/superres/super8/super8_best_from_mp2.mp4
frames= 7195 
width= 720 
height= 480 
fps= 59.94005994005994

Process File: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/super8_best_from_mp2-10VDSR.mp4
factor: 1.0 
width= 720 
height= 480 
bitrate= 4000k



 13%|█▎        | 905/7195 [00:33<03:54, 26.86it/s]


 25%|██▍       | 1796/7195 [01:06<03:22, 26.70it/s]


 37%|███▋      | 2684/7195 [01:40<02:50, 26.49it/s]


 50%|████▉     | 3572/7195 [02:13<02:14, 26.90it/s]


...

 74%|███████▍  | 5348/7195 [03:19<01:08, 26.88it/s]


 87%|████████▋ | 6236/7195 [03:52<00:35, 26.76it/s]


 99%|█████████▉| 7124/7195 [04:25<00:02, 26.88it/s]


100%|██████████| 7195/7195 [04:28<00:00, 26.79it/s]


True

In [46]:
fn = str(Path(infile).stem) + "-"+str(int(factor*10))+"VDSRdual.mp4"
dualfile_s = str(result_path/fn)
infile_s = str(infile)
outfile_s = str(outfile)
print("Input 1:",infile_s)
print("Input 2:",outfile_s)
print("Output:",dualfile_s)

Input 1: /media/SSD/superres/super8/super8_best_from_mp2.mp4
Input 2: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/super8_best_from_mp2-10VDSR.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/super8_best_from_mp2-10VDSRdual.mp4


In [47]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile_s+'"'} -i {'"'+outfile_s+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile_s+'"'}

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/media/SSD/conda_env/superres --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/media/SSD/su

### Comparisons

In [66]:
infile = Path('/media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Late 1890s - A Trip Through Paris France (speed corrected w added sound)-10.mp4')
outfile = Path('/media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Late 1890s - A Trip Through Paris France (speed corrected w added sound)-10VDSR.mp4')

In [67]:
fn = str(Path(infile).stem) + "comp.mp4"
dualfile_s = str(result_path/fn)
infile_s = str(infile)
outfile_s = str(outfile)
print("Input 1:",infile_s)
print("Input 2:",outfile_s)
print("Output:",dualfile_s)

Input 1: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Late 1890s - A Trip Through Paris France (speed corrected w added sound)-10.mp4
Input 2: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Late 1890s - A Trip Through Paris France (speed corrected w added sound)-10VDSR.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Late 1890s - A Trip Through Paris France (speed corrected w added sound)-10comp.mp4


In [68]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile_s+'"'} -i {'"'+outfile_s+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile_s+'"'}

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/media/SSD/conda_env/superres --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/media/SSD/su

### Comparisons

In [58]:
infile = Path('/media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-10.mp4')
outfile = Path('/media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-10VDSR.mp4')

In [59]:
fn = str(Path(infile).stem) + "comp.mp4"
dualfile_s = str(result_path/fn)
infile_s = str(infile)
outfile_s = str(outfile)
print("Input 1:",infile_s)
print("Input 2:",outfile_s)
print("Output:",dualfile_s)

Input 1: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-10.mp4
Input 2: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-10VDSR.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/8mm Film Restored 1939-1940-10comp.mp4


In [60]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile_s+'"'} -i {'"'+outfile_s+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile_s+'"'}

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/media/SSD/conda_env/superres --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/media/SSD/su

### Comparisons

In [61]:
infile = Path('/media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker-10.mp4')
outfile = Path('/media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker-10VDSR.mp4')

In [62]:
fn = str(Path(infile).stem) + "comp.mp4"
dualfile_s = str(result_path/fn)
infile_s = str(infile)
outfile_s = str(outfile)
print("Input 1:",infile_s)
print("Input 2:",outfile_s)
print("Output:",dualfile_s)

Input 1: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker-10.mp4
Input 2: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker-10VDSR.mp4
Output: /media/SSD/superres/Results/Superresolution train New Network with kernel predictor make predictions/Early 60s 8mm Film Scans Using the Wolverine Film2Digital Moviemaker-10comp.mp4


In [63]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile_s+'"'} -i {'"'+outfile_s+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile_s+'"'}

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/media/SSD/conda_env/superres --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/media/SSD/su

In [ ]:

outfile = result_path/"test-20.mp4"
superres_mp4(infile, outfile, mcombine,channels=3, factor=2.0,display=False,bitrate= "4000k")

In [ ]:
yt = YouTube('https://www.youtube.com/watch?v=k0lQHHiRLS4')

In [ ]:
yt.streams.filter(subtype='mp4').all()

In [ ]:
yt.streams.get_by_itag(134).download('/media/SSD/superres/video_data/demos')

In [ ]:
infile = "/media/SSD/superres/muffins30sec.mp4"
outfile = result_path/"muffins30sec-20.mp4"
superres_mp4(infile, outfile, mcombine,channels=3, factor=2.0,display=False,bitrate= "4000k")

In [ ]:
infile = "/media/SSD/superres/muffins30sec.mp4"
outfile = result_path/"muffins30sec-30.mp4"
superres_mp4(infile, outfile, mcombine,channels=3, factor=3.0,display=False,bitrate= "4000k")


In [ ]:
dualfile= str(result_path/"muffins30sec-30dual.mp4")
infile = str(infile)
outfile = str(outfile)

In [ ]:
# double scale for video x3 2 min
!ffmpeg -i {'"'+infile+'"'} -i {'"'+outfile+'"'} -t 120 -filter_complex "[0:v] scale=iw*2:ih*2, pad=2*iw:ih [left]; [1:v] scale=iw/1.5:ih/1.5 [right]; [left][right] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile+'"'}

In [ ]:
# original scale , for video x3
#!ffmpeg -i {'"'+infile+'"'} -i {'"'+outfile+'"'} -filter_complex "[0:v] scale=iw*1:ih*1, pad=2*iw:ih [left]; [1:v] scale=iw/3:ih/3 [right]; [left][right] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile+'"'}

In [ ]:
infile = "/media/SSD/superres/super8/super8_best_from_mp2.mp4"
outfile = result_path/"super8.mp4"
error_file = "/media/SSD/superres/error.txt"
superres_mp4(infile, outfile, mcombine,channels=3, factor=1.0,display=False,bitrate= "4000k",errorfile = error_file)

In [ ]:

dualfile = str(result_path/"super8dual.mp4")
infile = str(infile)
outfile = str(outfile)

In [ ]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile+'"'} -i {'"'+outfile+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile+'"'}

In [ ]:
# compare this model to the last one
infile = "/media/SSD/superres/Results/Superresolution train New Network upscale PRelu L1 Div2 training/super8.mp4"
outfile = str(result_path/"super8.mp4")
dualfile= str(result_path/"super8-tolast-dual.mp4")

In [ ]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile+'"'} -i {'"'+outfile+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile+'"'}

In [ ]:
infile = "/media/SSD/superres/super8/Old 8mm Reels_1.mp4"
outfile = str(result_path/"super8long.mp4")
error_file = "/media/SSD/superres/error.txt"
superres_mp4(infile, outfile, mcombine,channels=3, factor=1.0,display=False,bitrate= "4000k",errorfile = error_file)

In [ ]:
dualfile= str(result_path/"super8longdual.mp4")
infile = str(infile)
outfile = str(outfile)

In [ ]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile+'"'} -i {'"'+outfile+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile+'"'}

In [ ]:
infile = "/media/SSD/superres/super8/Old 8mm Reels_1.mp4"
outfile = str(result_path/"super8longcrop.mp4")
error_file = "/media/SSD/superres/error.txt"
superres_mp4(infile, outfile, mcombine,channels=3, factor=1.0,display=False,bitrate= "4000k",
             errorfile = error_file, center_crop = (680,450))

In [ ]:
dualfile= str(result_path/"super8longcropdual.mp4")
infile = str(infile)
outfile = str(outfile)

In [ ]:
# original scale , for film x1, removed un needed scaling
!ffmpeg -i {'"'+infile+'"'} -i {'"'+outfile+'"'} -filter_complex "[0:v]  pad=2*iw:ih [left];  [left][1:v] overlay=main_w/2:0" -b:v 4000k {'"'+dualfile+'"'}